# Basic Weather App

In [14]:
import requests

def get_weather(api_key, location):
    
    # OpenWeatherMap API endpoint
    api_url = "http://api.openweathermap.org/data/2.5/weather"
    
    # Parameters for the API request
    params = {
        'q': location,
        'appid': api_key,
        'units': 'metric'
    }
    
    try:
        
        # Make the API request
        response = requests.get(api_url, params = params)
        data = response.json()
        
        # Check if the request was successful (HTTP status code 200)
        if response.status_code == 200:
            return data
        
        else:
            # Print error message from the API response
            print(f"Error: {data['message']}")
            return None

    except Exception as e:
        
        # Handle any exceptions that might occur during the request
        print(f"Error : {e}")
        return None
        
def display_weather(weather_data):
    
    # Display weather information if available
    if weather_data:
        
        print(f"Location: {weather_data['name']}")
        print(f"Temperature: {weather_data['main']['temp']}°C")
        print(f"Humidity: {weather_data['main']['humidity']}%")
        print(f"Weather Conditions: {weather_data['weather'][0]['description']}")
        
    else:
        # Display a message if weather information is not available
        print("Weather information not available.")
        
def main():
    
    # Main function to run the Weather App
    print("Basic Weather App")
    api_key = "5b050d9e770a90d2cbc6960fba8845e5"
    location = input("Enter location (city or ZIP code): ")
    
    # Get weather data from the API
    weather_data = get_weather(api_key, location)
    
    # Display weather information
    display_weather(weather_data)
    
if __name__ =="__main__":
    # Run the main function when the script is executed
    main()
    

Basic Weather App
Enter location (city or ZIP code): 636001
Location: Salem
Temperature: 29.1°C
Humidity: 51%
Weather Conditions: haze


# Weather App

In [2]:
import tkinter as tk 
from tkinter import ttk
import requests
from PIL import Image, ImageTk
from geopy.geocoders import Nominatim

class WeatherApp():
    
    def __init__(self, master):
        
        # Initialize the WeatherApp instance
        self.master = master
        self.master.title("Weather App")
        
        # Variables to store location, weather data, and temperature unit
        self.location = tk.StringVar()
        self.weather_data = {}
        self.temperature_unit = "metric"
        
        # Set up the user interface
        self.setup_ui()
        
        
    def fetch_weather(self):
        
        # Fetch weather data based on user input or GPS location
        location = self.location.get()
        
        if not location:
            location = self.get_user_location()
            
        
        if location:
            
            # API request to OpenWeatherMap
            api_key = "5b050d9e770a90d2cbc6960fba8845e5"
            base_url = "http://api.openweathermap.org/data/2.5/weather"
            params = {
                    'q': location,
                    'appid': api_key,
                    'units': self.temperature_unit
                }
            
            try:
                # Make the API request
                response = requests.get(base_url, params = params)
                data = response.json()
                
                if response.status_code == 200:
                    # Update weather data and GUI
                    self.weather_data = data
                    self.update_gui()     
                else:
                    # Print error message from the API response
                    print(f"Error: {data['message']}")
                
            except Exception as e:
                # Handle exceptions during the request
                print(f"Error: {e}")
            
        else:
            print("Error! Please enter a location or enable GPS for automatic location.")
            
            
    def get_user_location(self):
        
        # Get user's location using geopy library
        geolocator = Nominatim(user_agent = "weather_app")
        
        try:
            # Get location information based on user's IP address
            location = geolocator.geocode("me")
            return f"{location.latitude},{location.longitude}"
        
        except Exception as e:
            print(f"Error getting location:{e}")
            return None
            
    def update_gui(self):
        
        # Update GUI elements with weather data
        temperature = self.weather_data['main']['temp']
        humidity = self.weather_data['main']['humidity']
        conditions = self.weather_data['weather'][0]['description']
        icon_code = self.weather_data['weather'][0]['icon']
        
        # Update labels with weather information
        self.temperature_label.config(text=f"Temperature: {temperature}°{self.get_temperature_unit_symbol()}")
        self.humidity_label.config(text=f" Humidity: {humidity}%")
        self.conditions_label.config(text=f"Conditions: {conditions}")

        # Load and display weather icon
        icon_url = f"http://openweathermap.org/img/wn/{icon_code}.png"
        icon_image = self.load_image_from_url(icon_url)
        self.weather_icon_label.configure(image = icon_image)
        self.weather_icon_reference = icon_image


    def clear_weather(self):
        
        # Clear weather data and reset GUI elements
        self.location.set("")
        self.temperature_label.config(text = "Temperature: ")
        self.humidity_label.config(text = "Humidity: ")
        self.conditions_label.config(text = "Conditions: ")
        self.weather_icon_label.configure(image = self.default_icon)
        self.weather_icon_reference = self.default_icon
        
        
    def load_image_from_url(self,url):
        
        # Load an image from a URL and return as PhotoImage
        response = requests.get(url, stream = True)
        image = Image.open(response.raw)
        image = image.resize((50, 50), Image.LANCZOS)
        return ImageTk.PhotoImage(image)
        
    def toggle_temperature_unit(self):
        
        # Toggle between metric and imperial temperature units
        if self.temperature_unit == "metric":
            self.temperature_unit = "imperial"
            
        else:
            self.temperature_unit = "metric"
            
        self.fetch_weather()
                                      
    def get_temperature_unit_symbol(self):
        
        # Get the temperature unit symbol (C or F)                              
        return "C" if self.temperature_unit == "metric" else "F"

    def setup_ui(self):
       
        # Label for getting Location
        location_label = ttk.Label(self.master, text = "Enter Location: ")
        location_label.grid(row = 0 , column = 0, padx = 10, pady = 10)
        
        location_entry = ttk.Entry(self.master, textvariable = self.location)
        location_entry.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        # Button to fetch weather
        fetch_button = ttk.Button(self.master, text = "Fetch Location", command = self.fetch_weather )
        fetch_button.grid(row = 0, column = 2, padx = 10, pady = 10)
        
        # Button to clear weather information
        clear_button = ttk.Button(self.master, text = "Clear", command = self.clear_weather)
        clear_button.grid(row = 0, column = 3, padx = 10, pady = 10)
        
        # Button to toggle temperature units
        unit_button = ttk.Button(self.master, text = "Toggle Unit", command = self.toggle_temperature_unit)
        unit_button.grid(row = 0, column = 4 , padx = 10, pady = 10)
        
        # Labels for displaying weather information
        self.temperature_label = ttk.Label(self.master, text = "Temperature: ")
        self.temperature_label.grid(row = 1, column = 0, padx = 10, pady = 10, columnspan = 3)
        
        self.humidity_label = ttk.Label(self.master, text = "Humidity: ")
        self.humidity_label.grid(row = 2, column = 0, padx = 10, pady = 10, columnspan = 3)
        
        self.weather_icon_label = ttk.Label(self.master)
        self.weather_icon_label.grid(row = 3, column = 0, padx = 10, pady = 10, columnspan = 3)
        
        self.conditions_label = ttk.Label(self.master, text = "Weather Condition: ")
        self.conditions_label.grid(row = 4, column = 0, padx = 10, pady = 10, columnspan = 3)
        
        # Load and set a default weather icon
        default_image = Image.open("./data/weather.png") 
        default_image = default_image.resize((50, 50), Image.LANCZOS)

        self.default_icon = ImageTk.PhotoImage(default_image)
        self.weather_icon_reference = None
    
        
def main():
    
    # Run the WeatherApp application
    root = tk.Tk()
    app = WeatherApp(root)
    root.mainloop()


if __name__ == "__main__":
    # Run the main function when the script is executed
    main()